In [1]:
# %% [markdown]
# ### Question 1: [IPO] Withdrawn IPOs by Company Type
# 
# **What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?**
# 
# From the withdrawn IPO list ([stockanalysis.com/ipos/withdrawn](https://stockanalysis.com/ipos/withdrawn/)), collect and process the data to find out which company type saw the most withdrawn IPO value.
# 
# #### Steps:
# 1. Use pandas.read_html() with the URL above to load the IPO withdrawal table into a DataFrame. 
#    *It is a similar process to Code Snippet 1 discussed at the livestream.*    You should get **99 entries**. 
# 2. Create a new column called Company Class, categorizing company names based on patterns like:
#    - “Acquisition Corp” or “Acquisition Corporation” → Acq.Corp
#    - “Inc” or “Incorporated” → Inc
#    - “Group” → Group
#    - “Ltd” or “Limited” → Limited
#    - “Holdings” → Holdings
#    - Others → Other
# 
#   *  Order: Please follow the listed order of classes and assign the first matched value (e.g., for 'shenni holdings limited', you assign the 'Limited' class).
# 
#   * Hint: make your function more robust by converting names to lowercase and splitting into words before matching patterns.
# 
# 3. Define a new field Avg. price by parsing the Price Range field (create a function and apply it to the Price Range column). Examples:
#    - '$8.00-$10.00' → 9.0  
#    - '$5.00' → 5.0  
#    - '-' → None
# 4. Convert Shares Offered to numeric, clean missing or invalid values.
# 5. Create a new column:  
#    Withdrawn Value = Shares Offered * Avg Price (**71 non-null values**)
# 6. Group by Company Class and calculate total withdrawn value.
# 7. **Answer**: Which class had the highest **total** value of withdrawals?

# %%
import pandas as pd
import requests
from io import StringIO

# Define the URL and headers to mimic a browser request
url = 'https://stockanalysis.com/ipos/withdrawn/'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
                   (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)
response.raise_for_status()

# Clean and safe way to parse HTML into a DataFrame
tables = pd.read_html(StringIO(response.text))
df = tables[0]
print(f"✅ Loaded {len(df)} rows.")



# %%
print(df.columns.tolist())


# %%
df['Company Class'] = df['Company Name'].apply(classify_company)


# %%
# Define function to classify company types based on keywords
def classify_company(name):
    name = name.lower()
    if 'acquisition corp' in name or 'acquisition corporation' in name:
        return 'Acq.Corp'
    elif 'inc' in name or 'incorporated' in name:
        return 'Inc'
    elif 'group' in name:
        return 'Group'
    elif 'ltd' in name or 'limited' in name:
        return 'Limited'
    elif 'holdings' in name:
        return 'Holdings'
    else:
        return 'Other'

# Apply the classification using 'Company Name' column
df['Company Class'] = df['Company Name'].apply(classify_company)


# %%
# Define function to parse and average price ranges
def parse_avg_price(price_range):
    if price_range == '-' or pd.isna(price_range):
        return None
    prices = price_range.replace('$', '').split('-')
    try:
        prices = [float(p) for p in prices]
        return sum(prices) / len(prices)
    except:
        return None

# Apply the function to create a new column
df['Avg Price'] = df['Price Range'].apply(parse_avg_price)


# %%
# Remove commas and convert Shares Offered to numeric
df['Shares Offered'] = pd.to_numeric(df['Shares Offered'].str.replace(',', ''), errors='coerce')


# %%
# Calculate Withdrawn Value as Shares Offered × Avg Price
df['Withdrawn Value'] = df['Shares Offered'] * df['Avg Price']

# Print how many non-null values were successfully calculated
print(f"✅ Non-null Withdrawn Value entries: {df['Withdrawn Value'].notnull().sum()}")  # Should be 71


# %%
# Group by Company Class and sum the Withdrawn Value
grouped = df.groupby('Company Class')['Withdrawn Value'].sum().sort_values(ascending=False)

# Show results
print("\n📊 Total Withdrawn IPO Value by Company Class (in millions):")
print(grouped)


# %%
# Find and print the top class and its total withdrawn value
max_class = grouped.idxmax()
max_value = grouped.max()
print(f"\n✅ The company class with the highest total withdrawn value is: {max_class} (${max_value:.2f} million)")


# %% [markdown]
# ### Question 2:   [IPO] Median Sharpe Ratio for 2024 IPOs (First 5 Months)
# 
# 
# **What is the median Sharpe ratio (as of 6 June 2025) for companies that went public in the first 5 months of 2024?**
# 
# The goal is to replicate the large-scale `yfinance` OHLCV data download and perform basic financial calculations on IPO stocks.
# 
# 
# #### Steps:
# 
# 1. Using the same approach as in Question 1, download the IPOs in 2024 from:  
#    [https://stockanalysis.com/ipos/2024/](https://stockanalysis.com/ipos/2024/)  
#    Filter to keep only those IPOs **before 1 June 2024** (first 5 months of 2024).  
#    ➤ You should have **75 tickers**.
# 
# 2.  Use **Code Snippet 7** to download daily stock data for those tickers (via `yfinance`).  
#    Make sure you understand how `growth_1d` ... `growth_365d`, and volatility columns are defined.  
#    Define a new column `growth_252d` representing growth after **252 trading days** (~1 year), in addition to any other growth periods you already track.
# 
# 
# 3. Calculate the Sharpe ratio assuming a risk-free rate of **4.5%**:
# 
#    ```python
#    stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']
#    ```
# 
#    ⚠️ **IMPORTANT** Please use the original version of annualized volatility calculation (it was later corrected to another formula):
#    ```python
#    stocks_df['volatility'] =   stocks_df['Close'].rolling(30).std() * np.sqrt(252)
#    ```
# 4. Filter the DataFrame to keep data only for the trading day:  
#    **‘2025-06-06’**
# 
#    Compute descriptive statistics (e.g., `.describe()`) for these columns:  
#    - `growth_252d`  
#    - `Sharpe`
# 
#    You should observe:  
#    - `growth_252d` is defined for **71 out of 75 stocks** (some IPOs are too recent or data starts later).  
#    - Median `growth_252d` is approximately **0.75** (indicating a 25% decline), while mean is about **1.15**, showing a bias towards high-growth companies pushing the average up.
# 
# 5. **Answer:**  
#    - What is the **median Sharpe ratio** for these 71 stocks?  
#    - Note: Positive `Sharpe` means growth exceeding the risk-free rate of 4.5%.  
#    - [Additional] Do you observe the **same top 10 companies** when sorting by `growth_252d` versus sorting by `Sharpe`?
# 

# %%
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
import requests
from bs4 import BeautifulSoup

# %%
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import StringIO  # For future compatibility

# STEP 1: Scrape IPO list from StockAnalysis.com
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Find the HTML table
table_html = str(soup.find("table"))

# Wrap in StringIO to avoid future warnings
df = pd.read_html(StringIO(table_html))[0]

# Rename and parse date
df.rename(columns={"IPO Date": "Date"}, inplace=True)
df["Date"] = pd.to_datetime(df["Date"])

# Filter IPOs before June 1, 2024
df = df[df["Date"] < "2024-06-01"]

# Clean symbol column
df = df[~df["Symbol"].isnull()]
df = df[df["Symbol"].str.isupper()]

# Extract ticker list
ipo_tickers = df["Symbol"].tolist()
print(f"✅ Tickers collected: {len(ipo_tickers)}")


# %%
import yfinance as yf
import numpy as np

# Initialize list to hold all ticker data
all_data = []

# Loop through each ticker
for i, ticker in enumerate(ipo_tickers):
    print(f"{i+1}/{len(ipo_tickers)}: Downloading {ticker}...")

    try:
        # Download 2024-2025 data from Yahoo Finance
        df = yf.download(ticker, start="2024-01-01", end="2025-06-07", progress=False)

        # Skip if data is empty
        if df.empty:
            print(f"⚠️ {ticker}: No data.")
            continue

        # Add identifier columns
        df["Ticker"] = ticker
        df["Date"] = df.index

        # Compute 1-year growth (252 trading days)
        df["growth_252d"] = df["Close"] / df["Close"].shift(252)

        # Compute annualized 30-day rolling volatility
        df["volatility"] = df["Close"].rolling(window=30).std() * np.sqrt(252)

        # Calculate Sharpe ratio using risk-free rate of 4.5%
        df["Sharpe"] = (df["growth_252d"] - 0.045) / df["volatility"]

        # Append result to main list
        all_data.append(df)

    except Exception as e:
        print(f"❌ Error with {ticker}: {e}")
        continue

# Combine all ticker data into one DataFrame
stocks_df = pd.concat(all_data).reset_index(drop=True)

print(f"\n✅ Completed data download for {len(stocks_df['Ticker'].unique())} tickers.")


# %%
print(stocks_df.columns.tolist())


# %%
print(isinstance(stocks_df.columns, pd.MultiIndex))


# %%
stocks_df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in stocks_df.columns]


# %%
print(stocks_df.columns.tolist())


# %%
stocks_df.rename(columns={
    "Date_": "Date",
    "growth_252d_": "growth_252d",
    "Sharpe_": "Sharpe"
}, inplace=True)


# %%
snapshot = stocks_df[stocks_df["Date"] == "2025-06-06"]
snapshot = snapshot.dropna(subset=["growth_252d", "Sharpe"])

print(f"📅 Valid records for 2025-06-06: {len(snapshot)} stocks")
print(snapshot[["growth_252d", "Sharpe"]].describe())

median_sharpe = snapshot["Sharpe"].median()
print(f"\n📌 Median Sharpe Ratio (as of 2025-06-06): {median_sharpe:.2f}")


# %%
top_growth = snapshot.sort_values("growth_252d", ascending=False).head(10)["Ticker_"].tolist()
top_sharpe = snapshot.sort_values("Sharpe", ascending=False).head(10)["Ticker_"].tolist()

overlap = set(top_growth) & set(top_sharpe)
print(f"🔁 Top 10 Overlap: {len(overlap)} tickers → {list(overlap)}")


# %%
# Sort and get top 10 by growth_252d
top_growth_df = snapshot.sort_values("growth_252d", ascending=False).head(10)

# Sort and get top 10 by Sharpe
top_sharpe_df = snapshot.sort_values("Sharpe", ascending=False).head(10)

# Display Top 10 by growth
print("🏆 Top 10 IPOs by 1-Year Growth:\n")
print(top_growth_df[["Ticker_", "growth_252d", "Sharpe"]].reset_index(drop=True))

# Display Top 10 by Sharpe
print("\n⭐ Top 10 IPOs by Sharpe Ratio:\n")
print(top_sharpe_df[["Ticker_", "Sharpe", "growth_252d"]].reset_index(drop=True))


# %% [markdown]
# ### Question 3: [IPO] ‘Fixed Months Holding Strategy’
# 
# **What is the optimal number of months (1 to 12) to hold a newly IPO'd stock in order to maximize average growth?**  
# (*Assume you buy at the close of the first trading day and sell after a fixed number of trading days.*)
# 
# 
# ---
# 
# #### Goal:
# Investigate whether holding an IPO stock for a fixed number of months after its first trading day produces better returns, using future growth columns.
# 
# ---
# 
# #### Steps:
# 
# 1. **Start from the existing DataFrame** from Question 2 (75 tickers from IPOs in the first 5 months of 2024).  
# 
#    Add **12 new columns**:  
#    `future_growth_1m`, `future_growth_2m`, ..., `future_growth_12m`  
#    *(Assume 1 month = 21 trading days, so growth is calculated over 21, 42, ..., 252 trading days)*  
#    This logic is similar to `historyPrices['growth_future_30d']` from **Code Snippet 7**, but extended to longer timeframes.
# 
# 2. **Determine the first trading day** (`min_date`) for each ticker.  
#    This is the earliest date in the data for each stock.
# 
# 3. **Join the data**:  
#    Perform an **inner join** between the `min_date` DataFrame and the future growth data on both `ticker` and `date`.  
#    ➤ You should end up with **75 records** (one per IPO) with all 12 `future_growth_...` fields populated.
# 
# 4. **Compute descriptive statistics** for the resulting DataFrame:  
#    Use `.describe()` or similar to analyze each of the 12 columns:  
#    - `future_growth_1m`  
#    - `future_growth_2m`  
#    - ...  
#    - `future_growth_12m`  
# 
# 5. **Determine the best holding period**:  
#    - Find the number of months **(1 to 12)** where the **average (mean)** future growth is **maximal**.  
#    - This optimal month shows an uplift of **>1%** compared to all others.  
#    - Still, the average return remains **less than 1** (i.e., expected return is less than doubling your investment).
# 
# 

# %%
# Create a fresh list to hold data per ticker
growth_frames = []

# Loop through each unique ticker
for ticker in stocks_df["Ticker_"].unique():
    df = stocks_df[stocks_df["Ticker_"] == ticker].copy()

    # Skip tickers without Close_<ticker> column
    close_col = f"Close_{ticker}"
    if close_col not in df.columns:
        print(f"⚠️ Skipping {ticker}: No '{close_col}' found.")
        continue

    # Base closing price
    close = df[close_col]

    # Add future growth columns: 1m (21d), 2m (42d), ..., 12m (252d)
    for m in range(1, 13):
        days = m * 21
        df[f"future_growth_{m}m"] = close.shift(-days) / close

    # Keep only first trading day
    first_row = df.sort_values("Date").iloc[[0]]
    growth_frames.append(first_row)

# Combine all first-day records into one DataFrame
first_day_growth_df = pd.concat(growth_frames).reset_index(drop=True)


# %%
# Columns for analysis
future_cols = [f"future_growth_{m}m" for m in range(1, 13)]

# Descriptive statistics
stats = first_day_growth_df[future_cols].describe().T
stats["mean_growth"] = first_day_growth_df[future_cols].mean().values

# Print summary
print(stats[["mean", "std", "mean_growth"]])

# Find best holding period
best_month = stats["mean_growth"].idxmax()
best_return = stats["mean_growth"].max()
print(f"\n📈 Best holding period: {best_month} → Avg growth: {best_return:.2f}")


# %% [markdown]
# ### Question 4: [Strategy] Simple RSI-Based Trading Strategy
# 
# 
# **What is the total profit (in $thousands) you would have earned by investing $1000 every time a stock was oversold (RSI < 25)?**
# 
# 
# ---
# 
# #### Goal:
# Apply a simple rule-based trading strategy using the **Relative Strength Index (RSI)** technical indicator to identify oversold signals and calculate profits.
# 
# ---
# 
# 
# #### Steps:
# 
# 1. **Run the full notebook from Lecture 2 (33 stocks)**  
#    - Ensure you can generate the merged DataFrame containing:  
#      - OHLCV data  
#      - Technical indicators  
#      - Macro indicators  
#    - Focus on getting **RSI** computed using **Code Snippets 8 and 9**.  
#    - This process is essential and will help during the capstone project.
# 
# 2. ⚠️ **IMPORTANT** Please use this file to solve the Home Assignment (**all next steps**)
#  
#    Download precomputed data using this snippet:
# 
#    ```python
#    import gdown
#    import pandas as pd
# 
#    file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
#    gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
#    df = pd.read_parquet("data.parquet", engine="pyarrow")
# 
# 3. **RSI Strategy Setup:**  
#    - RSI is already available in the dataset as a field.  
#    - The threshold for **oversold** is defined as `RSI < 25`.
# 
# 4. **Filter the dataset by RSI and date:**  
#    ```python
#    rsi_threshold = 25
#    selected_df = df[
#        (df['rsi'] < rsi_threshold) &
#        (df['Date'] >= '2000-01-01') &
#        (df['Date'] <= '2025-06-01')
#    ]
# 5. **Calculate Net Profit Over 25 Years:**  
#    - Total number of trades: **1568**  
#    - For each trade, you invest **$1000**  
#    - Use the 30-day forward return (`growth_future_30d`) to compute net earnings:  
#      ```python
#      net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()
#      ```
# 
#    - **Final Answer:**  
#      What is the **net income in $K** (i.e., in thousands of dollars) that could be earned using this RSI-based oversold strategy from 2000–2025?
# 
# 

# %%
pip install pyarrow


# %%
print(df.columns.tolist())


# %%
pip install ta


# %%
import pandas as pd
import ta

# Load your parquet file
df = pd.read_parquet("C:/Users/rahul/stock-markets-analytics-zoomcamp/02-dataframe-analysis/stocks_df_combined_2025_06_20.parquet.brotli", engine="pyarrow")

# Compute RSI for AAPL
df = df.sort_values("Date")
df["rsi_AAPL"] = ta.momentum.RSIIndicator(close=df["Close_AAPL"], window=14).rsi()

# 30-day future return
df["growth_future_30d_AAPL"] = df["Close_AAPL"].shift(-30) / df["Close_AAPL"]

# Filter RSI < 25 and date range
selected = df[
    (df["rsi_AAPL"] < 25) &
    (df["Date"] >= "2000-01-01") &
    (df["Date"] <= "2025-06-01")
]

# Calculate total net income from $1000 per trade
net_income = 1000 * (selected["growth_future_30d_AAPL"] - 1).sum()
print(f"📈 Total net income from RSI strategy on AAPL: ${net_income / 1000:.2f}K")


# %%
pip install gdown


# %%
import gdown
import pandas as pd

# Download the assignment dataset
file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)

# Load it
df = pd.read_parquet("data.parquet", engine="pyarrow")


# %%
import pandas as pd

df = pd.read_parquet("data.parquet", engine="pyarrow")

selected_df = df[
    (df['rsi'] < 25) &
    (df['Date'] >= '2000-01-01') &
    (df['Date'] <= '2025-06-01')
]

net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()
print(f"📌 Net income from RSI strategy: ${net_income / 1000:.2f}K")


# %% [markdown]
# Question 5. Q5. [Exploratory, Optional] Predicting a Positive-Return IPO (1 point)

# %% [markdown]
# To predict whether an IPO will yield a positive 1-year return, we can use a combination of price-based, company-level, technical, and macroeconomic features. Early performance indicators like first-day return, short-term volatility, and momentum (e.g., 30-day or 90-day growth) offer strong signals. Metadata such as the sector, IPO month, and exchange listing may capture structural or seasonal patterns. Technical indicators like RSI, MACD, and moving average crossovers help identify early price momentum or reversals. Finally, market conditions—such as S&P 500 trends, VIX levels, or interest rates—provide useful context about the broader economic environment at IPO time.

# %% [markdown]
# Classification problem with a Target: growth_252d > 1  → "Positive Return"
# 




SyntaxError: invalid syntax (487512650.py, line 477)